In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [2]:
#hiden cell
CLIENT_ID = '*********' # your Foursquare ID
CLIENT_SECRET = '**********' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *******
CLIENT_SECRET:*******


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Kyoto",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Kyoto', 'Kyoto')

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 35.03055245265477, 'lng': 135.7620153055578},
  'sw': {'lat': 35.01137622447278, 'lng': 135.74601260449722}},
 42)

In [7]:
d["geocode"]

{'what': '',
 'where': 'kyoto',
 'center': {'lat': 35.02107, 'lng': 135.75385},
 'displayString': 'Kyoto, Kyōto, Japan',
 'cc': 'JP',
 'geometry': {'bounds': {'ne': {'lat': 35.321201324462876,
    'lng': 135.87820434570298},
   'sw': {'lat': 34.87493515014652, 'lng': 135.55920410156298}}},
 'slug': 'kyoto-japan',
 'longId': '72057594039785846'}

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 42


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4cfc9ad520fe370448bc5af8',
  'name': 'Morpho Cafe (モルフォカフェ)',
  'location': {'address': '上京区皀莢町309',
   'crossStreet': '西堀川通中立売下る',
   'lat': 35.02354750323727,
   'lng': 135.75175213809223,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.02354750323727,
     'lng': 135.75175213809223}],
   'postalCode': '602-8242',
   'cc': 'JP',
   'city': '京都市',
   'state': '京都府',
   'country': '日本',
   'formattedAddress': ['上京区皀莢町309 (西堀川通中立売下る)',
    '京都市, 京都府',
    '602-8242',
    '日本']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'gr

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '上京区皀莢町309', 'crossStreet': '西堀川通中立売下る', 'lat': 35.02354750323727, 'lng': 135.75175213809223, 'labeledLatLngs': [{'label': 'display', 'lat': 35.02354750323727, 'lng': 135.75175213809223}], 'postalCode': '602-8242', 'cc': 'JP', 'city': '京都市', 'state': '京都府', 'country': '日本', 'formattedAddress': ['上京区皀莢町309 (西堀川通中立売下る)', '京都市, 京都府', '602-8242', '日本']}
{'address': '上京区春帯町345', 'crossStreet': '丸太町通新町上ル', 'lat': 35.0185886115978, 'lng': 135.75649095009277, 'labeledLatLngs': [{'label': 'display', 'lat': 35.0185886115978, 'lng': 135.75649095009277}], 'postalCode': '602-8026', 'cc': 'JP', 'city': '京都市', 'state': '京都府', 'country': '日本', 'formattedAddress': ['上京区春帯町345 (丸太町通新町上ル)', '京都市, 京都府', '602-8026', '日本']}
{'address': '中京区西竹屋町529', 'crossStreet': 'SONGBIRD DESIGN STORE. 2F', 'lat': 35.016192986241904, 'lng': 135.75193956317173, 'labeledLatLngs': [{'label': 'display', 'lat': 35.016192986241904, 'lng': 135.75193956317173}], 'postalCode': '604-0075', 'cc': 'JP', 'city': '京都市', 'st

,uid,name,shortname,address,postalcode,lat,lng
0,4cfc9ad520fe370448bc5af8,Morpho Cafe (モルフォカフェ),Vegetarian / Vegan,,602-8242,35.023548,135.751752
1,4d67c99f5c2ecbff17f12ce9,Point Pour Point 府庁前店,Desserts,,602-8026,35.018589,135.756491
2,4b909469f964a520db9033e3,Songbird Coffee,Café,,604-0075,35.016193,135.751940
3,4b7e8c41f964a52022f22fe3,拾得,Coffee Shop,,602-8125,35.019421,135.748991
4,585367369f25833d214bbce1,Time Dou (タイム堂),Café,,602-8023,35.018400,135.759287


In [12]:
Kyoto_center = d["geocode"]["center"]
Kyoto_center

{'lat': 35.02107, 'lng': 135.75385}

In [13]:
from folium import plugins


map_Kyoto = folium.Map(location=[35.02107, 135.75385], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Kyoto)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Kyoto.add_child(plugins.HeatMap(hm_data))

map_Kyoto

By spotting the clusters of items we can see which neighborhood has density of coffee business.

We will need a location where we can catch out customers from "hot" location we have picked up from the map and stay in a certain distance so as to lessen the competivity of business.

In [14]:
lat = 35.02107
lng = 135.75385
map_Tokyo = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Here It Is!!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Tokyo)
map_Tokyo